# Introduction


In this notebook we demostrate how to finetune **Llama3** (8B) model from Meta using **QLoRA** and **SFTTrainer** from **tlr**.

## What is Llama3?

Llama3 is the latest release of open-source LLMs from Meta, with features pretrained and instruction-fine-tuned language models with 8B and 70B parameters.

## What is LoRA?

LoRA stands for Low-Rank Adaptation. It is a method used to fine-tune large language models (LLMs) by freezing the weights of the LLM and injecting trainable rank-decomposition matrices. The number of trainable parameters during fine-tunning will decrease therefore considerably. According to LoRA paper, this number decreases 10,000 times, and the computational resources size decreases 3 times.


## What is QLoRA?

QLoRA builds on LoRA by incorporating quantization techniques to further reduce memory usage while maintaining, or even enhancing, model performance. With QLoRA it is possible to finetune a 70B parameter model that requires 36 GPUs with only 2!


## What is PEFT?

Parameter-efficient Fine-tuning (PEFT) is a technique used in Natural Language Processing (NLP) to improve the performance of pre-trained language models on specific downstream tasks. It involves reusing the pre-trained model’s parameters and fine-tuning them on a smaller dataset, which saves computational resources and time compared to training the entire model from scratch. PEFT achieves this efficiency by freezing some of the layers of the pre-trained model and only fine-tuning the last few layers that are specific to the downstream task.

## What is SFTTrainer?

SFT in SFTTrainer stands for supervised fine-tuning. The **trl** (Transformer Reinforcement Learning) library from HuggingFace provides a simple API to fine-tune models using SFTTrainer.

## What is UltraChat200k?  

UltraChat-200k is an invaluable resource for natural language understanding, generation and dialog system research. With 1.4 million dialogues spanning a variety of topics, this parquet-formatted dataset offers researchers four distinct formats to aid in their studies: test_sft, train_sft, train_gen and test_gen. More details [here](https://www.kaggle.com/datasets/thedevastator/ultrachat-200k-nlp-dataset).

## Inspiration

For this notebook, I took inspiration from several sources:
* [Efficiently fine-tune Llama 3 with PyTorch FSDP and Q-Lora](https://www.philschmid.de/fsdp-qlora-llama3)  
* [Fine-tuning LLMs using LoRA](https://medium.com/@rajatsharma_33357/fine-tuning-llama-using-lora-fb3f48a557d5)  
* [Fine-tuning Llama-3–8B-Instruct QLORA using low cost resources](https://medium.com/@avishekpaul31/fine-tuning-llama-3-8b-instruct-qlora-using-low-cost-resources-89075e0dfa04)  
* [Llama2 Fine-Tuning with Low-Rank Adaptations (LoRA) on Gaudi 2 Processors](https://eduand-alvarez.medium.com/llama2-fine-tuning-with-low-rank-adaptations-lora-on-gaudi-2-processors-52cf1ee6ce11)  

# Install and import libraries

In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U transformers
!pip install -q -U peft
!pip install -q -U accelerate
!pip install -q -U datasets
!pip install -q -U trl

In [2]:
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [3]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
wandb_key = user_secrets.get_secret("wandb_api")
import wandb
! wandb login $wandb_key

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [4]:
import os
import torch
from time import time
from datasets import load_dataset
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
)
from trl import SFTTrainer,setup_chat_format

2024-05-04 15:09:58.217425: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-04 15:09:58.217525: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-04 15:09:58.353803: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Initialize model


The model used is:

* **Model**: Llama3  
* **Framework**: Transformers   
* **Size**: 8B   
* **Type**: 8b-chat-hf (hf stands for HuggingFace). 
* **Version**: V1  

In [5]:
model_id = "/kaggle/input/llama-3/transformers/8b-chat-hf/1"

We are using quantization (with BitsAndBytes).

In [6]:
compute_dtype = torch.bfloat16
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True)

We define the model configuration, the model (using AutoModelForCausalLM) and the tokenizer (using AutoTokenizer).

In [7]:
time_start = time()

model_config = AutoConfig.from_pretrained(
    model_id,
    trust_remote_code=True,
    max_new_tokens=1024
)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
time_end = time()
print(f"Prepare model, tokenizer: {round(time_end-time_start, 3)} sec.")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Prepare model, tokenizer: 128.518 sec.


In [8]:
model, tokenizer = setup_chat_format(model, tokenizer)
model = prepare_model_for_kbit_training(model)

# Prepare the dataset   


We will use 10K rows from the `ultrachat_200k` database.

In [9]:
dataset_name = "HuggingFaceH4/ultrachat_200k"
dataset = load_dataset(dataset_name, split="train_sft")
dataset = dataset.shuffle(seed=42).select(range(10000))

def format_chat_template(row):
    chat = tokenizer.apply_chat_template(row["messages"], tokenize=False)
    return {"text":chat}

processed_dataset = dataset.map(
    format_chat_template,
    num_proc= os.cpu_count(),
)

dataset = processed_dataset.train_test_split(test_size=0.01)

Generating train_sft split:   0%|          | 0/207865 [00:00<?, ? examples/s]

Generating test_sft split:   0%|          | 0/23110 [00:00<?, ? examples/s]

Generating train_gen split:   0%|          | 0/256032 [00:00<?, ? examples/s]

Generating test_gen split:   0%|          | 0/28304 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/10000 [00:00<?, ? examples/s]

# Training

## Training configuration   

We set the rank for LoRA to 4, to reduce the number of trainable parameters.

In [10]:
peft_config = LoraConfig(
        lora_alpha=64,
        lora_dropout=0.05,
        r=4,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules= ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",]
)

In [11]:
training_arguments = TrainingArguments(
        output_dir="./results_llama3_sft/",
        evaluation_strategy="steps",
        do_eval=True,
        optim="paged_adamw_8bit",
        per_device_train_batch_size=8,
        gradient_accumulation_steps=2,
        per_device_eval_batch_size=8,
        log_level="debug",
        save_steps=1,
        logging_steps=1,
        learning_rate=8e-6,
        eval_steps=1,
        max_steps=20,
        num_train_epochs=20,
        warmup_steps=3,
        lr_scheduler_type="linear",
)

## Training process

In [12]:
os.environ["WANDB_DISABLED"] = "false"

In [13]:
trainer = SFTTrainer(
        model=model,
        train_dataset=dataset['train'],
        eval_dataset=dataset['test'],
        peft_config=peft_config,
        dataset_text_field="text",
        max_seq_length=512,
        tokenizer=tokenizer,
        args=training_arguments,
)

trainer.train()

Map:   0%|          | 0/9900 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

You have loaded a model on multiple GPUs. `is_model_parallel` attribute will be force-set to `True` to avoid any unexpected behavior such as device placement mismatching.
max_steps is given, it will override any value given in num_train_epochs
Currently training with a batch size of: 8
***** Running training *****
  Num examples = 9,900
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 2
  Total optimization steps = 20
  Number of trainable parameters = 10,485,760
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: gpreda. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.16.6
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240504_151325-c4skxe5p
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run grievous-master-6
wandb: ⭐️ View proj

Step,Training Loss,Validation Loss
1,1.771800,1.882431
2,2.015200,1.865535
3,1.591800,1.835319
4,2.000800,1.796772
5,1.739000,1.767073
6,1.999700,1.744272
7,1.719400,1.726580
8,1.764300,1.712652
9,1.563000,1.701362
10,1.748000,1.692108


***** Running Evaluation *****
  Num examples = 100
  Batch size = 8
Saving model checkpoint to ./results_llama3_sft/checkpoint-1
/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in /kaggle/input/llama-3/transformers/8b-chat-hf/1 - will assume that the vocabulary was not modified.
  warnings.warn(
tokenizer config file saved in ./results_llama3_sft/checkpoint-1/tokenizer_config.json
Special tokens file saved in ./results_llama3_sft/checkpoint-1/special_tokens_map.json
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
***** 

TrainOutput(global_step=20, training_loss=1.7243618607521056, metrics={'train_runtime': 6447.7198, 'train_samples_per_second': 0.05, 'train_steps_per_second': 0.003, 'total_flos': 7387957124136960.0, 'train_loss': 1.7243618607521056, 'epoch': 0.03231017770597738})

# Conclusions

By reducing the batch_size, max_seq_length and the LoRA rank, we managed to run the Llama3 fine-tunning with QLoRA in the Kaggle environment.
